# Segmenting and Clustering Neighborhoods in Toronto

1/ Scrape the postal codes data from Wikipedia, clean and process the data  
2/ Get the latitude and the longitude coordinates of each neighborhood  
3/ Explore and cluster the neighborhoods in Toronto:
    - using the Foursquare API
    - get the most common venue categories in each neighborhood
    - group the neighborhoods into clusters using the k-means clustering algorithm
    - visualize the neighborhoods in Toronto and their emerging clusters using the Folium library.

## Download all dependencies and libraries

In [1]:
!pip install beautifulsoup4
!pip install lxml

from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# Part 1: Scrape and clean the postal code data from Wikipedia
#### Web scraping is done using BeautifulSoup.

In [2]:
#source URL
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#using the 'lmxl' parser
soup = BeautifulSoup(source,'lxml')

#check the title to confirmt he page has been scraped properly
print(soup.title)

#read all the tables in pandas dataframes
list_of_df = pd.read_html(str(soup.table))

#extract the first table
df=list_of_df[0]

df.head()

<title>List of postal codes of Canada: M - Wikipedia</title>


0             1                 2
0  Postal Code       Borough     Neighbourhood
1          M1A  Not assigned      Not assigned
2          M2A  Not assigned      Not assigned
3          M3A    North York         Parkwoods
4          M4A    North York  Victoria Village

#### Data processing and cleaning
First we label the columns of the dataframe with the headers located in first row (index 0)

In [3]:
# Label the columns and drop first row
df = pd.DataFrame(df.values[1:], columns=df.iloc[0])
df.head()

0 Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront

Then we remove the row which don't have an assigned borough

In [4]:
# Remove Not Assigned rows
df = df[df.Borough != 'Not assigned']
df.head()

0 Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

Then we combine neighbourhoods with the same postal code

In [5]:
# Combine Neigbourhoods with same postal code
df = df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
df.head()

Postal Code      Borough                           Neighbourhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae

Finally we replace the names of neighbourhoods which are 'not assigned' with the borough name

In [6]:
# Replacing names of neighbourhoods which are 'Not assigned' with names of Borough
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned',df['Borough'], df['Neighbourhood'])
df.head()

Postal Code      Borough                           Neighbourhood
0         M1B  Scarborough                          Malvern, Rouge
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough       Guildwood, Morningside, West Hill
3         M1G  Scarborough                                  Woburn
4         M1H  Scarborough                               Cedarbrae

Printing the shape of the cleaned up dataframe

In [7]:
print("Shape: ", df.shape)

Shape:  (103, 3)



# Part 2: Get the latitude and longitude coordinates of each neighbourhood
Import the geospatial master data in a dataframe

In [8]:
#import the coordinates from csv file
df_coor = pd.read_csv('https://cocl.us/Geospatial_data')
df_coor.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Merge the two dataframes to get each neighbourhood's coordinates

In [9]:
# Merge on 'Postal Code' column, no need to change column names as they are the same
df_neighbor = pd.merge(df,df_coor,on='Postal Code')
df_neighbor.head()

Postal Code      Borough                           Neighbourhood   Latitude  \
0         M1B  Scarborough                          Malvern, Rouge  43.806686   
1         M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2         M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3         M1G  Scarborough                                  Woburn  43.770992   
4         M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476



# Part 3: Cluster and explore the neighbourhoods of Toronto

### 3.1. Use geopy to get coordinates of Toronto

In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 3.2. Select only the neighbourhoods that contain "Toronto"

In [11]:
df_toronto = df_neighbor[df_neighbor['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto.head()

Postal Code          Borough                   Neighbourhood   Latitude  \
0         M4E     East Toronto                     The Beaches  43.676357   
1         M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2         M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3         M4M     East Toronto                 Studio District  43.659526   
4         M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

### 3.3. Map of the selected Toronto neighbourhoods

In [12]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.4. Foursquare credentials

In [13]:
CLIENT_ID = 'V3MITK25TB4Q5CDXCBSCE2TC3AHWA4VPH3M3C3WFTC3HXCDW' # your Foursquare ID
CLIENT_SECRET = '2NYGJCR55OCQ1KITMMBEEGS14HAYLIMCYY43COGFFFUDEGGQ' # your Foursquare Secret
VERSION = '20200730' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: V3MITK25TB4Q5CDXCBSCE2TC3AHWA4VPH3M3C3WFTC3HXCDW
CLIENT_SECRET:2NYGJCR55OCQ1KITMMBEEGS14HAYLIMCYY43COGFFFUDEGGQ


### 3.5. Explore the neighbourhoods & venues categories

Get the Top 100 venues of each neighbourhood within a radius of 500 meters, by creating a function that:  
* extracts name, latitude and longitude of a neighbourhood from our dataset
* inputs the client credentials, name, latitude, longitude to the Foursquare API to explore venues within 500m radius of the neighbourhood
* retrieves Venue data from the Json code that Foursquare API outputs
* creates a dataframe of all the relevant information required to analyze the venues

In [14]:
radius = 500
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We apply that function to our dataset

In [15]:
toronto_venues = getNearbyVenues(names = df_toronto['Neighbourhood'],
                                   latitudes = df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

Check the resulting Venues dataset

In [16]:
print(toronto_venues.shape)
toronto_venues.head(10)

(1634, 7)


Neighbourhood  Neighbourhood Latitude  \
0                   The Beaches               43.676357   
1                   The Beaches               43.676357   
2                   The Beaches               43.676357   
3                   The Beaches               43.676357   
4  The Danforth West, Riverdale               43.679557   
5  The Danforth West, Riverdale               43.679557   
6  The Danforth West, Riverdale               43.679557   
7  The Danforth West, Riverdale               43.679557   
8  The Danforth West, Riverdale               43.679557   
9  The Danforth West, Riverdale               43.679557   

   Neighbourhood Longitude                               Venue  \
0               -79.293031                   Glen Manor Ravine   
1               -79.293031  The Big Carrot Natural Food Market   
2               -79.293031                 Grover Pub and Grub   
3               -79.293031                       Upper Beaches   
4               -79.352188                       MenEssentials   
5               -79.352188                        Dolce Gelato   
6               -79.352188                            Pantheon   
7               -79.352188                     Cafe Fiorentina   
8               -79.352188              Louis Cifer Brew Works   
9               -79.352188                          La Diperie   

   Venue Latitude  Venue Longitude      Venue Category  
0       43.676821       -79.293942               Trail  
1       43.678879       -79.297734   Health Food Store  
2       43.679181       -79.297215                 Pub  
3       43.680563       -79.292869        Neighborhood  
4       43.677820       -79.351265      Cosmetics Shop  
5       43.677773       -79.351187      Ice Cream Shop  
6       43.677621       -79.351434    Greek Restaurant  
7       43.677743       -79.350115  Italian Restaurant  
8       43.677663       -79.351313             Brewery  
9       43.677702       -79.352265      Ice Cream Shop

There are 1634 venues in our dataset (as of July 31st 2020).  
We check how many venues were selected in each neighbourhood

In [17]:
toronto_venues[['Neighbourhood','Venue']].groupby('Neighbourhood').count()

Venue
Neighbourhood                                            
Berczy Park                                            57
Brockton, Parkdale Village, Exhibition Place           25
Business reply mail Processing Centre, South Ce...     19
CN Tower, King and Spadina, Railway Lands, Harb...     15
Central Bay Street                                     63
Christie                                               17
Church and Wellesley                                   75
Commerce Court, Victoria Hotel                        100
Davisville                                             33
Davisville North                                        8
Dufferin, Dovercourt Village                           16
First Canadian Place, Underground city                100
Forest Hill North & West, Forest Hill Road Park         4
Garden District, Ryerson                              100
Harbourfront East, Union Station, Toronto Islands     100
High Park, The Junction South                          24
India Bazaar, The Beaches West                         18
Kensington Market, Chinatown, Grange Park              65
Lawrence Park                                           3
Little Portugal, Trinity                               47
Moore Park, Summerhill East                             3
North Toronto West, Lawrence Park                      21
Parkdale, Roncesvalles                                 14
Queen's Park, Ontario Provincial Government            34
Regent Park, Harbourfront                              45
Richmond, Adelaide, King                              100
Rosedale                                                4
Roselawn                                                2
Runnymede, Swansea                                     33
St. James Town                                         88
St. James Town, Cabbagetown                            44
Stn A PO Boxes                                         97
Studio District                                        41
Summerhill West, Rathnelly, South Hill, Forest ...     16
The Annex, North Midtown, Yorkville                    22
The Beaches                                             4
The Danforth West, Riverdale                           42
Toronto Dominion Centre, Design Exchange              100
University of Toronto, Harbord                         35

Find out how many different venue categories are existing in our dataset:

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


### 3.6. Analyze each neighborhood

Let's find the occurence of each category by using one-hot encoding, which will return '1' if a category exists in the neighbourhood, and '0' if it doesn't exist.

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1634, 234)


Neighbourhood  Afghan Restaurant  Airport  \
0                   The Beaches                  0        0   
1                   The Beaches                  0        0   
2                   The Beaches                  0        0   
3                   The Beaches                  0        0   
4  The Danforth West, Riverdale                  0        0   

   Airport Food Court  Airport Lounge  Airport Service  Airport Terminal  \
0                   0               0                0                 0   
1                   0               0                0                 0   
2                   0               0                0                 0   
3                   0               0                0                 0   
4                   0               0                0                 0   

   American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0                    0             0         0            0           0   
1                    0             0         0            0           0   
2                    0             0         0            0           0   
3                    0             0         0            0           0   
4                    0             0         0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Workshop  \
0                    0                 0                   0              0   
1                    0                 0                   0              0   
2                    0                 0                   0              0   
3                    0                 0                   0              0   
4                    0                 0                   0              0   

   BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  \
0          0           0           0       0     0    0                 0   
1          0           0           0       0     0    0                 0   
2          0           0           0       0     0    0                 0   
3          0           0           0       0     0    0                 0   
4          0           0           0       0     0    0                 0   

   Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  Beer Store  \
0                   0      0                0         0           0   
1                   0      0                0         0           0   
2                   0      0                0         0           0   
3                   0      0                0         0           0   
4                   0      0                0         0           0   

   Belgian Restaurant  Bistro  Boat or Ferry  Bookstore  Brazilian Restaurant  \
0                   0       0              0          0                     0   
1                   0       0              0          0                     0   
2                   0       0              0          0                     0   
3                   0       0              0          0                     0   
4                   0       0              0          0                     0   

   Breakfast Spot  Brewery  Bubble Tea Shop  Building  Burger Joint  \
0               0        0                0         0             0   
1               0        0                0         0             0   
2               0        0                0         0             0   
3               0        0                0         0             0   
4               0        0                0         0             0   

   Burrito Place  Bus Line  Butcher  Café  Cajun / Creole Restaurant  \
0              0         0        0     0                          0   
1              0         0        0     0                          0   
2              0         0        0     0                          0   
3              0         0        0     0                          0   
4              0         0        0     0                          0   

   Camera Store  Ca

Group rows by neighbourhood and taking the mean of occurrence as frequency

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

Neighbourhood  Afghan Restaurant  \
0                                         Berczy Park           0.000000   
1        Brockton, Parkdale Village, Exhibition Place           0.000000   
2   Business reply mail Processing Centre, South C...           0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
4                                  Central Bay Street           0.000000   
5                                            Christie           0.000000   
6                                Church and Wellesley           0.013333   
7                      Commerce Court, Victoria Hotel           0.000000   
8                                          Davisville           0.000000   
9                                    Davisville North           0.000000   
10                       Dufferin, Dovercourt Village           0.000000   
11             First Canadian Place, Underground city           0.000000   
12    Forest Hill North & West, Forest Hill Road Park           0.000000   
13                           Garden District, Ryerson           0.000000   
14  Harbourfront East, Union Station, Toronto Islands           0.000000   
15                      High Park, The Junction South           0.000000   
16                     India Bazaar, The Beaches West           0.000000   
17          Kensington Market, Chinatown, Grange Park           0.000000   
18                                      Lawrence Park           0.000000   
19                           Little Portugal, Trinity           0.000000   
20                        Moore Park, Summerhill East           0.000000   
21                  North Toronto West, Lawrence Park           0.000000   
22                             Parkdale, Roncesvalles           0.000000   
23        Queen's Park, Ontario Provincial Government           0.000000   
24                          Regent Park, Harbourfront           0.000000   
25                           Richmond, Adelaide, King           0.000000   
26                                           Rosedale           0.000000   
27                                           Roselawn           0.000000   
28                                 Runnymede, Swansea           0.000000   
29                                     St. James Town           0.000000   
30                        St. James Town, Cabbagetown           0.000000   
31                                     Stn A PO Boxes           0.000000   
32                                    Studio District           0.000000   
33  Summerhill West, Rathnelly, South Hill, Forest...           0.000000   
34                The Annex, North Midtown, Yorkville           0.000000   
35                                        The Beaches           0.000000   
36                       The Danforth West, Riverdale           0.000000   
37           Toronto Dominion Centre, Design Exchange           0.000000   
38                     University of Toronto, Harbord           0.000000   

     Airport  Airport Food Court  Airport Lounge  Airport Service  \
0   0.000000            0.000000        0.000000              0.0   
1   0.000000            0.000000        0.000000              0.0   
2   0.000000            0.000000        0.000000              0.0   
3   0.066667            0.066667        0.133333              0.2   
4   0.000000            0.000000        0.000000              0.0   
5   0.000000            0.000000        0.000000              0.0   
6   0.000000            0.000000        0.000000              0.0   
7   0.000000            0.000000        0.000000              0.0   
8   0.000000            0.000000        0.000000              0.0   
9   0.000000            0.000000        0.000000              0.0   
10  0.000000            0.000000        0.000000              0.0   
11  0.000000            0.000000        0.000000              0.0   
12  0.000000            0.000000        0.000000              0.0   
13  0.000000            0.000000 

Create a function to sort the top venues in descending order

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a dataframe of the top 10 venues by neighbourhood

In [26]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# Labelling columns as 1st, 2nd and so on
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind])) # for 1st, 2nd, 3rd
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) ### for 4th, 5th,...

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns) ## assign column names we just created to a new dataframe
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']## add neighborhoods column

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

Neighbourhood 1st Most Common Venue  \
0                                         Berczy Park           Coffee Shop   
1        Brockton, Parkdale Village, Exhibition Place                  Café   
2   Business reply mail Processing Centre, South C...    Light Rail Station   
3   CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                  Central Bay Street           Coffee Shop   
5                                            Christie         Grocery Store   
6                                Church and Wellesley           Coffee Shop   
7                      Commerce Court, Victoria Hotel           Coffee Shop   
8                                          Davisville           Pizza Place   
9                                    Davisville North           Pizza Place   
10                       Dufferin, Dovercourt Village              Pharmacy   
11             First Canadian Place, Underground city           Coffee Shop   
12    Forest Hill North & West, Forest Hill Road Park         Jewelry Store   
13                           Garden District, Ryerson        Clothing Store   
14  Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
15                      High Park, The Junction South                  Café   
16                     India Bazaar, The Beaches West             Pet Store   
17          Kensington Market, Chinatown, Grange Park                  Café   
18                                      Lawrence Park                  Park   
19                           Little Portugal, Trinity                   Bar   
20                        Moore Park, Summerhill East            Playground   
21                  North Toronto West, Lawrence Park        Clothing Store   
22                             Parkdale, Roncesvalles             Gift Shop   
23        Queen's Park, Ontario Provincial Government           Coffee Shop   
24                          Regent Park, Harbourfront           Coffee Shop   
25                           Richmond, Adelaide, King           Coffee Shop   
26                                           Rosedale                  Park   
27                                           Roselawn                Garden   
28                                 Runnymede, Swansea           Coffee Shop   
29                                     St. James Town                  Café   
30                        St. James Town, Cabbagetown           Coffee Shop   
31                                     Stn A PO Boxes           Coffee Shop   
32                                    Studio District                  Café   
33  Summerhill West, Rathnelly, South Hill, Forest...           Coffee Shop   
34                The Annex, North Midtown, Yorkville        Sandwich Place   
35                                        The Beaches          Neighborhood   
36                       The Danforth West, Riverdale      Greek Restaurant   
37           Toronto Dominion Centre, Design Exchange           Coffee Shop   
38                     University of Toronto, Harbord                  Café   

            2nd Most Common Venue      3rd Most Common Venue  \
0              Seafood Restaurant                     Bakery   
1                          Bakery             Breakfast Spot   
2                     Yoga Studio              Auto Workshop   
3                  Airport Lounge           Airport Terminal   
4                  Sandwich Place                       Café   
5                            Café                       Park   
6             Japanese Restaurant           Sushi Restaurant   
7                            Café                 Restaurant   
8                    Dessert Shop             Sandwich Place   
9                           Hotel          Food & Drink Shop   
10                         Bakery  Middle Eastern Restaurant   
11                           Café                      Hotel   
12                          Trail              

### 3.7. Cluster the neighbourhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [27]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:38] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 1, 0,
       0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Lets put all data together including the cluster labels, neighbourhoods and top 10 venues

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

Postal Code          Borough                   Neighbourhood   Latitude  \
0         M4E     East Toronto                     The Beaches  43.676357   
1         M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2         M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3         M4M     East Toronto                 Studio District  43.659526   
4         M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0          Neighborhood     Health Food Store   
1 -79.352188               0      Greek Restaurant    Italian Restaurant   
2 -79.315572               0             Pet Store         Movie Theater   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               2                  Park              Bus Line   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Pub                 Trail     Electronics Store   
1           Coffee Shop        Ice Cream Shop            Restaurant   
2         Burrito Place        Sandwich Place  Fast Food Restaurant   
3               Brewery             Gastropub                Bakery   
4           Swim School           Yoga Studio                 Diner   

         6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0  Eastern European Restaurant   Dumpling Restaurant            Donut Shop   
1       Furniture / Home Store          Liquor Store     Indian Restaurant   
2           Italian Restaurant     Fish & Chips Shop            Restaurant   
3          American Restaurant     Convenience Store    Seafood Restaurant   
4                  Event Space  Ethiopian Restaurant     Electronics Store   

         9th Most Common Venue 10th Most Common Venue  
0                Deli / Bodega       Doner Restaurant  
1                          Spa              Juice Bar  
2                   Steakhouse       Sushi Restaurant  
3               Sandwich Place            Cheese Shop  
4  Eastern European Restaurant    Dumpling Restaurant

Visualize the clusters using folium

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.8. Examining the clusters

#### Cluster 1 (Red)

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

Postal Code           Borough  \
0          M4E      East Toronto   
1          M4K      East Toronto   
2          M4L      East Toronto   
3          M4M      East Toronto   
5          M4P   Central Toronto   
6          M4R   Central Toronto   
7          M4S   Central Toronto   
9          M4V   Central Toronto   
11         M4X  Downtown Toronto   
12         M4Y  Downtown Toronto   
13         M5A  Downtown Toronto   
14         M5B  Downtown Toronto   
15         M5C  Downtown Toronto   
16         M5E  Downtown Toronto   
17         M5G  Downtown Toronto   
18         M5H  Downtown Toronto   
19         M5J  Downtown Toronto   
20         M5K  Downtown Toronto   
21         M5L  Downtown Toronto   
24         M5R   Central Toronto   
25         M5S  Downtown Toronto   
26         M5T  Downtown Toronto   
27         M5V  Downtown Toronto   
28         M5W  Downtown Toronto   
29         M5X  Downtown Toronto   
30         M6G  Downtown Toronto   
31         M6H      West Toronto   
32         M6J      West Toronto   
33         M6K      West Toronto   
34         M6P      West Toronto   
35         M6R      West Toronto   
36         M6S      West Toronto   
37         M7A  Downtown Toronto   
38         M7Y      East Toronto   

                                        Neighbourhood   Latitude  Longitude  \
0                                         The Beaches  43.676357 -79.293031   
1                        The Danforth West, Riverdale  43.679557 -79.352188   
2                      India Bazaar, The Beaches West  43.668999 -79.315572   
3                                     Studio District  43.659526 -79.340923   
5                                    Davisville North  43.712751 -79.390197   
6                   North Toronto West, Lawrence Park  43.715383 -79.405678   
7                                          Davisville  43.704324 -79.388790   
9   Summerhill West, Rathnelly, South Hill, Forest...  43.686412 -79.400049   
11                        St. James Town, Cabbagetown  43.667967 -79.367675   
12                               Church and Wellesley  43.665860 -79.383160   
13                          Regent Park, Harbourfront  43.654260 -79.360636   
14                           Garden District, Ryerson  43.657162 -79.378937   
15                                     St. James Town  43.651494 -79.375418   
16                                        Berczy Park  43.644771 -79.373306   
17                                 Central Bay Street  43.657952 -79.387383   
18                           Richmond, Adelaide, King  43.650571 -79.384568   
19  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752   
20           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576   
21                     Commerce Court, Victoria Hotel  43.648198 -79.379817   
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678   
25                     University of Toronto, Harbord  43.662696 -79.400049   
26          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049   
27  CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420   
28                                     Stn A PO Boxes  43.646435 -79.374846   
29             First Canadian Place, Underground city  43.648429 -79.382280   
30                                           Christie  43.669542 -79.422564   
31                       Dufferin, Dovercourt Village  43.669005 -79.442259   
32                           Little Portugal, Trinity  43.647927 -79.419750   
33       Brockton, Parkdale Village, Exhibition Place  43.636847 -79.428191   
34                      High Park, The Junction South  43.661608 -79.464763   
35                             Parkdale, Roncesvalles  43.648960 -79.456325   
36                                 Runnymede, Swansea  43.651571 -79.484450   
37        Queen's Park, Ontario Provincial Government  43.662301 -79.389494   
38  Business reply mail Processing Centre, South C...  4

This is the biggest cluster. Lots of coffee shops, restaurants, sandwich places. These are lively neighbourhoods with lots of eateries and places to go out.

#### Cluster 2 (Purple)

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1] # purple

Postal Code           Borough                Neighbourhood   Latitude  \
8          M4T   Central Toronto  Moore Park, Summerhill East  43.689574   
10         M4W  Downtown Toronto                     Rosedale  43.679563   

    Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
8  -79.383160               1            Playground                  Park   
10 -79.377529               1                  Park            Playground   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
8           Tennis Court           Yoga Studio      Department Store   
10                 Trail           Yoga Studio      Department Store   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
8   Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   
10  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   9th Most Common Venue 10th Most Common Venue  
8    Dumpling Restaurant             Donut Shop  
10   Dumpling Restaurant             Donut Shop

This look to be more of an outdoor and sports area with parks, playgrounds ,tennsi courts, trail and yoga studio.

#### Cluster 3 (Blue)

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2] #blue

Postal Code          Borough  Neighbourhood  Latitude  Longitude  \
4         M4N  Central Toronto  Lawrence Park  43.72802  -79.38879   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
4               2                  Park              Bus Line   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
4           Swim School           Yoga Studio                 Diner   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
4           Event Space  Ethiopian Restaurant     Electronics Store   

         9th Most Common Venue 10th Most Common Venue  
4  Eastern European Restaurant    Dumpling Restaurant

This cluster is located near a big park

#### Cluster 4 (Green)

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3] #green

Postal Code          Borough Neighbourhood   Latitude  Longitude  \
22         M5N  Central Toronto      Roselawn  43.711695 -79.416936   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
22               3                Garden          Home Service   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
22          Dessert Shop    Falafel Restaurant           Event Space   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
22  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   9th Most Common Venue 10th Most Common Venue  
22   Dumpling Restaurant             Donut Shop

This area specific has a garden as distinctive feature.

#### Cluster 5 (Orange)

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4] #orange

Postal Code          Borough  \
23         M5P  Central Toronto   

                                      Neighbourhood   Latitude  Longitude  \
23  Forest Hill North & West, Forest Hill Road Park  43.696948 -79.411307   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
23               4         Jewelry Store                 Trail   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
23              Bus Line      Sushi Restaurant           Yoga Studio   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
23                 Diner           Event Space  Ethiopian Restaurant   

   9th Most Common Venue       10th Most Common Venue  
23     Electronics Store  Eastern European Restaurant

One of the farthest clusters, looks like quiet so it could be a more residential area.